In [34]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm
from itertools import combinations, permutations
import time

In [35]:
data_A = np.loadtxt('../data/TSPA.csv', delimiter=';').astype(np.int64)
data_B = np.loadtxt('../data/TSPB.csv', delimiter=';').astype(np.int64)
data_C = np.loadtxt('../data/TSPC.csv', delimiter=';').astype(np.int64)
data_D = np.loadtxt('../data/TSPD.csv', delimiter=';').astype(np.int64)

In [36]:
def create_cost_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2) + cost.reshape(1, -1)).round().astype(np.int64)

def create_dist_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    #cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2)).round().astype(np.int64)

In [37]:
cost_matrix_A = create_cost_matrix(data_A)

In [38]:
cost_matrix_A

array([[  84, 2032, 2098, ..., 4159, 3783, 1514],
       [1633,  483, 2398, ..., 3349, 2266,  817],
       [ 720, 1419, 1462, ..., 3640, 3149,  964],
       ...,
       [2782, 2371, 3641, ..., 1461, 2908, 2554],
       [2558, 1440, 3302, ..., 3060, 1309, 1773],
       [1234,  936, 2062, ..., 3651, 2718,  364]])

In [39]:
cost_matrix_B = create_cost_matrix(data_B)

In [40]:
cost_matrix_C = create_cost_matrix(data_C)

In [41]:
cost_matrix_D = create_cost_matrix(data_D)

In [42]:
dist_matrix_A = create_dist_matrix(data_A)
dist_matrix_B = create_dist_matrix(data_B)
dist_matrix_C = create_dist_matrix(data_C)
dist_matrix_D = create_dist_matrix(data_D)

In [43]:
def plot(data, solution):
    data_ordered = np.array([data[i] for i in solution])
    all_data = np.array([data[i] for i in range(200)])

    plt.figure(figsize=(10, 10), dpi=80)

    plt.scatter(data_ordered[:,0], data_ordered[:,1], s=data_ordered[:,2]/data_ordered[:,2].max()*200, c='b')
    plt.scatter(all_data[:,0], all_data[:,1], s=all_data[:,2]/all_data[:,2].max()*200, c='b')
    plt.plot(data_ordered[:,0], data_ordered[:,1], 'y-')
    plt.plot([data_ordered[0,0], data_ordered[-1,0]], [data_ordered[0,1], data_ordered[-1,1]], 'y-')
    plt.show()

In [44]:
def calculate_performance(cycle, cost_matrix):
    total_sum = 0
    for i in range(len(cycle)-1):
        total_sum += cost_matrix[cycle[i], cycle[i+1]]
    total_sum += cost_matrix[cycle[-1], cycle[0]]
    return total_sum

In [45]:
np.arange(0, 100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [46]:
@jit()
def random_solution(cost_matrix, limit=100):
    random_solution_list = np.arange(0, 100)
    np.random.shuffle(random_solution_list)
    return random_solution_list[:limit]

solution = random_solution(cost_matrix_A, 100)
print(solution)

[59 42 33 53 76 34 43 73 28 36 82 67 22  2 80 58 95 71  5 20 89 84  1 98
 54 92 31 55 25 64 12 38 45 23 26 51 83 40 93 81 41 39  6 14  8 66 18 27
 17 63 65 35 60 13 74 52 68 94 46 30 88 99 37 96 77 57 29 78 97  7 15 87
 24 75 85 32 19 90 10  3 48 62 56 61 49 69 91 70  0 44  4  9 21 47 72 11
 86 79 16 50]


In [47]:
# %%timeit
# random_solution(cost_matrix_A)

In [48]:
@jit()
def greedy_cycle(cost_matrix, current_id, limit=100):
    all_ids = set(list(range(0,len(cost_matrix))))
    all_ids.remove(current_id)
    solution = [current_id]
    
    for _ in range(1):
        min_val = 99999
        min_id = -1
        for next_id in all_ids:
            if cost_matrix[current_id][next_id] < min_val:
                min_val = cost_matrix[current_id][next_id]
                min_id = next_id
        solution.append(min_id)
        all_ids.remove(min_id)
        current_id = min_id
    
    while len(solution) < limit:
        min_delta = 99999
        min_id = -1
        insert_id = -1
        for i in range(len(solution)-1):
            for next_id in all_ids:
                delta = cost_matrix[solution[i]][next_id] + cost_matrix[next_id][solution[i+1]] - cost_matrix[solution[i]][solution[i+1]]
                if delta < min_delta:
                    min_delta = delta
                    min_id = next_id
                    insert_id = i
        for next_id in all_ids:
            delta = cost_matrix[solution[-1]][next_id] + cost_matrix[next_id][solution[0]] - cost_matrix[solution[-1]][solution[0]]
            if delta < min_delta:
                min_delta = delta
                min_id = next_id
                insert_id = i
        solution.insert(insert_id+1, min_id)
        all_ids.remove(min_id)

    return np.array(solution)

In [49]:
calculate_performance(greedy_cycle(cost_matrix_A, 2), cost_matrix_A)

77328

In [50]:
# %%timeit
# greedy_cycle(cost_matrix_A, 0)

In [51]:
# @jit()
# def calculate_closest_neighbours(cost_matrix, n_neighbours=10):
#     def extract_nodes(ll):
#         return [x[1] for x in ll]
    
#     diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
#     np.fill_diagonal(diag_matrix, np.inf)
    
#     new_cost_matrix = cost_matrix + diag_matrix

#     closest_neighbours = dict()
#     for i in range(len(new_cost_matrix)):
#         closest_neighbours[i] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
#     return closest_neighbours

In [52]:
jit(nopython=True)
def calculate_closest_neighbours(cost_matrix, n_neighbours=10):
    def extract_nodes(ll):
        return [x[1] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix + diag_matrix

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[i] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [53]:
# calculate_closest_neighbours(cost_matrix_A, 10)

In [54]:
jit(nopython=True)
def calculate_closest_neighbours_solution(cost_matrix, solution, n_neighbours=10):
    def extract_nodes(ll):
        return [solution[x[1]] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix[solution, :][:, solution] + diag_matrix[solution, :][:, solution]

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[solution[i]] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [55]:
calculate_closest_neighbours(cost_matrix_A)

{0: [19, 178, 149, 114, 4, 55, 164, 50, 76, 43],
 1: [177, 75, 189, 41, 199, 48, 152, 11, 119, 4],
 2: [4, 114, 77, 0, 43, 149, 75, 175, 167, 91],
 3: [178, 0, 164, 19, 128, 59, 132, 143, 96, 55],
 4: [114, 77, 0, 43, 149, 19, 121, 91, 75, 175],
 5: [135, 95, 98, 112, 169, 72, 6, 51, 167, 8],
 6: [98, 141, 66, 186, 72, 156, 135, 172, 79, 190],
 7: [74, 163, 195, 55, 62, 113, 117, 22, 53, 32],
 8: [95, 169, 80, 26, 135, 48, 119, 31, 189, 75],
 9: [167, 75, 189, 135, 177, 186, 4, 114, 101, 119],
 10: [132, 55, 74, 113, 163, 185, 128, 195, 96, 59],
 11: [48, 152, 75, 177, 189, 106, 119, 1, 160, 26],
 12: [98, 94, 95, 72, 31, 6, 135, 169, 73, 112],
 13: [26, 48, 8, 95, 119, 169, 106, 75, 189, 135],
 14: [111, 95, 80, 31, 8, 98, 94, 169, 72, 135],
 15: [117, 55, 195, 22, 74, 53, 62, 79, 163, 108],
 16: [48, 75, 152, 11, 189, 177, 119, 106, 130, 1],
 17: [75, 189, 177, 167, 4, 135, 114, 119, 199, 1],
 18: [55, 117, 195, 22, 74, 53, 163, 62, 113, 79],
 19: [178, 0, 149, 164, 114, 4, 43, 128, 

In [56]:
calculate_closest_neighbours_solution(cost_matrix_A, [1,2,3,4,5,6,7,8,9,10,11,12,13])

{1: [11, 4, 8, 6, 9, 2, 5, 12, 13, 7],
 2: [4, 1, 6, 11, 8, 9, 5, 12, 7, 10],
 3: [4, 10, 6, 1, 7, 2, 11, 8, 9, 5],
 4: [1, 2, 6, 11, 8, 9, 5, 12, 7, 3],
 5: [6, 8, 12, 4, 1, 11, 9, 13, 2, 7],
 6: [8, 12, 4, 5, 1, 11, 9, 7, 2, 13],
 7: [6, 4, 10, 1, 8, 3, 2, 5, 12, 11],
 8: [11, 6, 1, 12, 5, 4, 13, 9, 2, 7],
 9: [4, 1, 6, 8, 11, 5, 2, 12, 13, 7],
 10: [4, 7, 6, 3, 1, 2, 8, 11, 9, 5],
 11: [1, 8, 4, 6, 13, 5, 9, 12, 2, 7],
 12: [6, 8, 5, 4, 11, 1, 9, 13, 2, 7],
 13: [8, 11, 1, 6, 4, 5, 12, 9, 2, 7]}

In [57]:
pre = calculate_closest_neighbours_solution(cost_matrix_A, list(range(100)))

In [58]:
solution = list(range(15)) + list(range(100, 115))

In [59]:
# %%timeit
# calculate_closest_neighbours_solution(cost_matrix_A, solution)

In [60]:
def calculate_change_before(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_2-1]] -
            ( cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_1]] + cost_matrix[enhanced_solution[position_2-1], enhanced_solution[position_2]] )
            )

def calculate_change_after(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[(position_1+1)%len(enhanced_solution)], enhanced_solution[(position_2+1)%len(enhanced_solution)]] -
            ( cost_matrix[enhanced_solution[position_1], enhanced_solution[(position_1+1)%len(enhanced_solution)]] + cost_matrix[enhanced_solution[position_2], enhanced_solution[(position_2+1)%len(enhanced_solution)]] )
            )

In [61]:
def calculate_change_inter(enhanced_solution, cost_matrix, position_1, node_closest):
    return (cost_matrix[enhanced_solution[position_1], node_closest] + cost_matrix[node_closest, enhanced_solution[(position_1+2)%len(enhanced_solution)]] -
           (cost_matrix[enhanced_solution[position_1], enhanced_solution[(position_1+1)%len(enhanced_solution)]] + cost_matrix[enhanced_solution[(position_1+1)%len(enhanced_solution)], enhanced_solution[(position_1+2)%len(enhanced_solution)]])
           )

In [62]:
import time

In [63]:
# @jit()
def enhance_solution(initial_solution, closest_neighbours, distance_matrix, cost_matrix):
    start_all = time.time()
    enhanced_solution = list(initial_solution)
    previous_solution = 0
    current_solution = calculate_performance(initial_solution, cost_matrix)
    
    total_time_rest = 0
    total_time_edges = 0
    total_time_inter = 0
    while True:
        best_type = None

        ## Edges
        previous_solution = current_solution
        best_delta_edges = 0                  # e.g. -10
        best_edges_positions = None     # e.g. [1, 10]
        best_type = None                # before or after

        best_delta_inter = 0
        best_positions_inter = None

        best_operation = None

        neighbours_positions = {node: enhanced_solution.index(node) for node in enhanced_solution}
        
        start_rest = time.time()
        for position in range(len(initial_solution)):
            for neighbour in closest_neighbours[enhanced_solution[position]]:
                ## Intra edges
                if neighbour in enhanced_solution:
                    start = time.time()
                    position_neighbour = neighbours_positions[neighbour]
                    
                    if abs(position - position_neighbour) == 1: continue
                        
                    
                    for swap_type in ('before', 'after'):
                        
                        if swap_type == 'before':
                            delta = calculate_change_before(enhanced_solution, distance_matrix, position_1=position, position_2=position_neighbour)
                            
                        elif swap_type == 'after':
                            delta = calculate_change_after(enhanced_solution, distance_matrix, position_1=position, position_2=position_neighbour)
                        
                        if delta < best_delta_edges:
                            best_delta_edges = delta
                            best_edges_positions = [position, position_neighbour]
                            best_type = swap_type
                    end = time.time()
                    total_time_edges += end - start
                ##Inter
                else:
                    start = time.time()
                    delta = calculate_change_inter(enhanced_solution, cost_matrix, position_1=position, node_closest=neighbour)
                    if delta < best_delta_inter:
                        best_delta_inter = delta
                        best_positions_inter = [position, neighbour]
                    end = time.time()
                    total_time_inter += end - start 

        end_rest = time.time()
        total_time_rest += end_rest - start_rest
           
        
        if any([best_delta_inter != 0, best_delta_edges != 0]):
            if best_delta_edges < best_delta_inter:
                best_operation = 'edges'
            else:
                best_operation = 'inter'
        elif all([best_delta_inter == 0, best_delta_edges == 0]):
            end_all = time.time()
            return enhanced_solution, total_time_edges, total_time_inter, end_all - start_all, total_time_rest

        #Edges
        if best_operation == 'edges':
            if best_delta_edges == 0:
                print("XD")
                return enhanced_solution, total_time_edges, total_time_inter
            else:
                if best_type == 'before':
                    if best_edges_positions[0] < best_edges_positions[1]:
                        enhanced_solution = enhanced_solution[:best_edges_positions[0]] + enhanced_solution[best_edges_positions[0]:best_edges_positions[1]][::-1] + enhanced_solution[best_edges_positions[1]:]
                    else:
                        enhanced_solution = enhanced_solution[best_edges_positions[1]:best_edges_positions[0]] + list(enhanced_solution[best_edges_positions[0]:] + enhanced_solution[:best_edges_positions[1]])[::-1]
                elif best_type == 'after':
                    if best_edges_positions[0] < best_edges_positions[1]:
                        enhanced_solution = enhanced_solution[:(best_edges_positions[0]+1)%len(enhanced_solution)] + enhanced_solution[(best_edges_positions[0]+1)%len(enhanced_solution):(best_edges_positions[1]+1)][::-1] + enhanced_solution[(best_edges_positions[1]+1):]
                    else:
                        enhanced_solution = enhanced_solution[best_edges_positions[1]+1:best_edges_positions[0]+1] + list(enhanced_solution[(best_edges_positions[0]+1):] + enhanced_solution[:best_edges_positions[1]+1])[::-1]
        #Inter
        else:
            enhanced_solution[(best_positions_inter[0]+1)%len(enhanced_solution)] = best_positions_inter[1]
            pass
        
                
        current_solution = calculate_performance(enhanced_solution, cost_matrix)
        assert current_solution < previous_solution

        
        


In [64]:
from tqdm import tqdm
import sys

In [65]:
def start_enhancement(starting_solution_type: str, cost_matrix, distance_matrix, n_neighbours=10):
    total_costs = []
    total_cost_random = 0
    total_time = 0
    total_time_edges = 0
    total_time_inter = 0
    total_time_rest = 0
    for i in tqdm(range(len(distance_matrix))):
        # print(i)
        if starting_solution_type == 'greedy':
            initial_solution = greedy_cycle(cost_matrix, i, limit=100)
        elif starting_solution_type == 'random':
            initial_solution = random_solution(cost_matrix, limit=100)
        else:
            return 'Bad type'
        initial_cost = calculate_performance(initial_solution, cost_matrix)
        total_cost_random += initial_cost

        closest_neighbours = calculate_closest_neighbours(distance_matrix, n_neighbours=n_neighbours)
        enhanced_solution, time_edges, time_inter, time_all, time_rest = enhance_solution(initial_solution, closest_neighbours, distance_matrix, cost_matrix)
        total_costs.append(calculate_performance(enhanced_solution, cost_matrix))
        total_time += time_all
        total_time_edges += time_edges
        total_time_inter += time_inter
        total_time_rest += time_rest

        ## Uncomment those lines and delete tqdm for it to work properly (useful for time improvement checking) 
        # sys.stdout.write("Total time (edges, inter, rest, everything) (%f, %f, %f, %f) Progress (%d / 200)  \r" % (total_time_edges, total_time_inter, total_time_rest, total_time, i+1) )
        # sys.stdout.flush()

    total_costs = np.array(total_costs)
    return (total_costs.mean(), total_costs.min(), total_costs.max()) #, total_cost_random / len(cost_matrix), total_time, total_time / len(cost_matrix)
        
        

        


In [66]:
start_enhancement('random', cost_matrix=cost_matrix_C, distance_matrix=dist_matrix_C, n_neighbours=10)

100%|██████████| 200/200 [02:38<00:00,  1.26it/s]


(51988.605, 49698, 55180)

In [67]:
start_enhancement('random', cost_matrix=cost_matrix_D, distance_matrix=dist_matrix_D, n_neighbours=10)

100%|██████████| 200/200 [02:44<00:00,  1.21it/s]


(49526.27, 46260, 53677)